In [1]:
import pandas as pd
import numpy as np
import matplotlib
import datetime as DT
import time
import pickle
%matplotlib inline
def expand_columns(data):  pd.set_option('display.max_columns',len(data.columns))
def describe2(data):
    print("COUNT", "UNIQUE", "#_NULL","DATA_TYPE","COLUMN_NAME : FIRST ITEM", sep="\t")
    for column in data:
        print(data[column].count(),data[column].nunique(),data[column].isnull().sum(), str(data[column].dtypes)+"\t",str(column)+" :",data[column].iloc[0],sep="\t")


In [2]:
#### Load Data #######
data = pd.read_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\registrant_data.csv", low_memory=False,encoding = "ISO-8859-1")
data["Registrant ID"] = data["Registrant ID"].fillna("FILL_NULL_w_USER_ID:"+data["User ID"])

In [3]:
#Gender handling
data["Gender"] = data['Registrant Gender'].copy()
data['Gender'] = data['Gender'].replace(to_replace=["Not specified"], value=np.nan)
data['Gender'] = data['Gender'].replace(to_replace=["f","F",'f ?'], value="Female")
data['Gender'] = data['Gender'].replace(to_replace=["m","M",'m?'], value="Male")


In [4]:
#new registrant IDs
# data = data.rename(columns = {'Registrant ID':"Original Registrant ID"})
# data["Registrant ID"] = data.groupby(["Registrant First Name","Registrant Last Name", "Registrant Birthdate","Gender"]).grouper.group_info[0].astype(str)
# data["Registrant ID"] =  np.where(data['Registrant ID'].str.contains("-"),data["Original Registrant ID"],data["Registrant ID"])
# data["Registrant ID"] = data["Registrant ID"].fillna("FILL_NULL_w_USER_ID:"+data["User ID"])

In [5]:
# data["Gender2"] = data["Gender"].copy()
# data = pd.get_dummies(data, columns=["Gender2"],dummy_na=False, prefix="",prefix_sep="")
# gb = data.copy()
# gb = gb.groupby("Registrant ID", as_index=False).mean()
# gb = pd.merge(data,gb, how="left", on="Registrant ID")
# gb.to_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\gb.csv")

# print(data["Original Registrant ID"].nunique())
# print(data["Registrant ID"].nunique())
# data["Registrant ID"].nunique() - data["Original Registrant ID"].nunique()

In [6]:
# describe2(data)

In [7]:
#format date variables
#coerce date will drop dates not in proper format
# .dt.date will convert back to date
data["Registrant Birthdate"] = pd.to_datetime(data["Registrant Birthdate"],format='%Y-%m-%d',errors='coerce') 
data["event_date"] = pd.to_datetime(data["event_date"],format='%Y-%m-%d',errors='coerce')#.dt.date
data["timestamp_registered"] = pd.to_datetime(data["timestamp_registered"],format='%Y-%m-%d',errors='coerce')
data["Registration Date"] = pd.to_datetime(data["timestamp_registered"].copy().dt.date)
data["Year"]=(data["event_date"].dt.year.astype(str)).str.replace('.0$', '').astype(float)

#create minute result time variable as float
data["result_time_Minutes"] = pd.DatetimeIndex(data["result_time"])
data["result_time_Minutes"] = data["result_time_Minutes"].dt.hour *60 + data["result_time_Minutes"].dt.minute +  data["result_time_Minutes"].dt.second / 60

#create minute result pace variable as float
data["results_net_pace_Minutes"] = pd.DatetimeIndex(data["results_net_pace"])
data["results_net_pace_Minutes"] = data["results_net_pace_Minutes"].dt.hour *60 + data["results_net_pace_Minutes"].dt.minute +  data["results_net_pace_Minutes"].dt.second / 60


In [8]:
# Age handling
now = pd.Timestamp(DT.datetime.now())
#create age variables
data['Age'] = (now - data["Registrant Birthdate"]).astype('<m8[Y]')
data["Age at Event"] = (data["event_date"] - data["Registrant Birthdate"]).astype('<m8[Y]')
print((data["Age at Event"] < 1).value_counts())
print((data["Age at Event"] > 80).value_counts())
#remove birthdays less than 1 year or more than 80
data['Age'] = np.where((data['Age'] < 1) | (data['Age'] > 80), np.nan, data["Age"])
data["Age at Event"] = np.where((data['Age at Event'] < 1) | (data['Age at Event'] > 80), np.nan, data["Age at Event"])

# data["Registrant Birthdate"] = data["Registrant Birthdate"] #.dt.date #drop time so uploads into SAS as date, not datetime variable
data["event_date"] = pd.to_datetime(data["event_date"]) #.dt.date
# data[["Registrant Birthdate",'Age',"Age at Event"]].describe(include="all")



False    923756
True       1436
Name: Age at Event, dtype: int64
False    924964
True        228
Name: Age at Event, dtype: int64


In [9]:
#Event categorization and Event distance/location handling

#create dictionary of event dates
cityEvent = data[["Registration Event Name","event_date"]].dropna()
cityEvent = cityEvent.set_index("Registration Event Name")["event_date"].to_dict()
data["event_date"] = data["Registration Event Name"].map(cityEvent) #fills in missing dates

data["Registration Category"] = data["Registration Category"].str.upper().str.strip() #removes white spaces and capitalizes
data["Kids"] = data["Registration Category"].str.extract(r'(YOUTH|JUNIOR|FAMILY|CHILD|KIDS)') #Kid or family marker
data["Kids"] = data["Kids"].str.replace(r'(YOUTH|JUNIOR|FAMILY|CHILD|KIDS)', "KIDS") #for dummy variable later
    #Catch 1.5 Mile Family Walk misclassified as 5 MILE
data["Category"] = data["Registration Category"].str.extract(r'(15K|5K|10K|5 MILE|10 MILE|HALF MARATHON|SEASON PASS|DONATION|RAFFLE|CONCERT|KIDS RACE)')
data["Category"] = np.where(data['Registration Category'].str.contains("(AQUABIKE|DUATHLON|TRI|OLYMPIC|SPRINT|TEAM)"),'TRI/OTHER',data["Category"])
data["Category"] = np.where(data['Registration Category'].str.contains("TRIPLE NICKEL"),"TRIPLE NICKEL",data["Category"])
data["Category"] = np.where(data['Registration Event Name'].str.contains("Goodie Bag"), "GOODIE BAG",data["Category"])
data["Category"] = np.where(data['Registration Event Name'].str.contains("Training"), "TRAINING",data["Category"])
data["Category"] = np.where(data['Registration Event Name'].str.contains("(Bike Valet|Parking)"), "VALET/PARKING",data["Category"])
data["Category"] = np.where(data["Registration Category"].str.contains("WALK"), "WALK",data["Category"])
#extract new kids to categorical
#location
data["event_location"] = np.where(data['Registration Event Name'].str.contains("Bangs Lake Multisport Festival"), "Wauconda, IL",data["event_location"])
data["event_location"] = np.where(data['Registration Event Name'].str.contains("2009 Terrapin 5K|Bucktown"), "Chicago, IL",data["event_location"])
data["event_location"] = np.where(data['Registration Event Name'].str.contains("North Shore"), "Highland Park, IL",data["event_location"])
data["event_location"] = np.where(data['Registration Event Name'].str.contains("Big Foot Triathlon"), "Lake Geneva, WI",data["event_location"])
#distance
data["distance_qty"] = np.where(data['Category'].str.contains("HALF MARATHON"), 21,data["distance_qty"])
data["distance_qty"] = np.where(data['Category'].str.contains("5K"), 5,data["distance_qty"])
data["distance_qty"] = np.where(data['Category'].str.contains("15K"), 15,data["distance_qty"])
data["distance_qty"] = np.where(data['Category'].str.contains("10 MILE"), 16,data["distance_qty"])
data["distance_qty"] = np.where(data['Category'].str.contains("5 MILE"), 8,data["distance_qty"])
data["distance_qty"] = np.where(data['Category'].str.contains("TRIPLE NICKEL"), 13,data["distance_qty"])
data['distance_unit'] = "kilometers"

data["Event"] = data["Registration Event Name"].str.upper().str.extract('(HOT CHOCOLATE|NORTH SHORE TURKEY TROT|CINCO DE MILER|BTN BIG 10K|FLEET FEET SPORTS SOLDIER FIELD 10 MILE|ROCK THE NIGHT 5K|BUCKTOWN 5K|HUMBOLDT PARK 5K|NORTH SHORE HALF MARATHON|BIG FOOT TRIATHLON|WALK FOR LITTLE CITY|TERRAPIN|TRAINING|BANGS LAKE MULTISPORT FESTIVAL|SOUTH SHORE TRIATHLON)')
data["Event"] = data["Event"].fillna("MISCELLANEOUS")

data["event_date"] = data["event_date"].fillna(pd.to_datetime(data["timestamp_registered"].dt.date))


C:\Users\patrick.carey\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:13: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
C:\Users\patrick.carey\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [10]:
print(data[(data["Event"]=="MISCELLANEOUS")]["Registration Category"].unique())
data[(data["Event"]=="MISCELLANEOUS")]["Registration Event Name"].unique()#["Registration Event Name"].unique()#.str.contains("Goodie Bag")]["Registration Event Name"].unique()


['TRI CAMP' 'THE PAMPERED RIDE (CHICAGO)' 'HITCHHIKE (CHICAGO)'
 'THE PAMPERED RIDE (DEERFIELD)' "WOMEN'S SPRINT" "WOMEN'S SPRINT - ATHENA"
 'DONATION' 'RAFFLE' '2014 5K SEASON PASS' '2014 DISTANCE SEASON PASS'
 '2015 SEASON PASS' 'TURKEY TROT' 'ROCK THE NIGHT' 'BUCKTOWN 5K SWAP'
 'HOT CHOCOLATE 15K/5K HOODIE' 'BUCKTOWN 5K JACKET' '2016 SEASON PASS'
 'LAS VEGAS 15K MEDAL' 'SEATTLE 15K MEDAL']


array(['2011 RAM Tri Camp', '2011 Bike Valet Service',
       '2011 She Bangs Tri', 'Help Mona Purdy', '2014 Season Pass',
       '2015 Season Pass', 'Goodie Bag Mail Out', '2016 Season Pass',
       'Extra Item: Mail Out'], dtype=object)

In [11]:
import geopy
import geopy.distance
locations = pd.read_table("C:\\Users\\patrick.carey\\Downloads\\US.txt",encoding = "ISO-8859-1", low_memory=False, 
                          names=["country","zip","city","full_state","state","county", "n","n1","n2","latitude","longitude","n3"])
locations["zip"] = locations["zip"].astype(int).astype(str)
locations['GPS'] = tuple(zip(locations.latitude, locations.longitude))
locations['event_location'] = locations['city'] + ', ' + locations['state']
GPS_City = locations.set_index('event_location')['GPS'].to_dict()
GPS_City["St. Louis, MO"] =(38.7023,-90.3644)
GPS_City["Austin, Tx"] =(30.3264,-97.7713)
GPS_City["Washington DC"] =(38.9122,-77.0177)
GPS_Zip = locations.set_index('zip')['GPS'].to_dict()
GPS_Lat = locations.set_index('zip')['latitude'].to_dict()
GPS_Long = locations.set_index('zip')['longitude'].to_dict()
data['city_coord'] = data['event_location'].map(GPS_City)
data['zip_coord'] = data['Registrant Zipcode'].map(GPS_Zip)
data['zip_coord_latitude'] = data['Registrant Zipcode'].map(GPS_Lat)
data['zip_coord_longitude'] = data['Registrant Zipcode'].map(GPS_Long)

data["Distance_To_Event(miles)"] = data.apply(lambda x: geopy.distance.distance(x['city_coord'], x['zip_coord']).miles, axis=1)

In [12]:
weather = pd.ExcelFile(r"C:\Users\patrick.carey\Desktop\RamRacing\2016-05-25_Weather.xlsx")
weatherDict ={}
for tab in weather.sheet_names:
    print(tab.replace(" ","*").replace("*","").partition(",")[0])
    #add each excel tab to the dictionary with name printed
    weatherDict[tab.replace(" ","").replace(".","").partition(",")[0]]=pd.DataFrame(weather.parse(tab))
    #add location column
#     weatherDict[tab.replace(" ","").replace(".","").partition(",")[0]].insert(0, "Location", tab)
    #add 30 day rolling average of temperature
    weatherDict[tab.replace(" ","").replace(".","").partition(",")[0]]["Prev_30d_Mean_Temp"] = pd.rolling_mean(weatherDict[tab.replace(" ","").replace(".","").partition(",")[0]]["Mean TemperatureF"], window=30)
    

# data["event_location"].unique() #list of all cities, weather only includes events with more than one year of race data

#move all weather data into one dataframe for merging
weather = pd.DataFrame(columns=weatherDict["Chicago"].keys())
# weather = pd.DataFrame(weatherDict["Chicago"].keys())
for dic in weatherDict:
    temp = pd.DataFrame.from_dict(weatherDict[dic])
    weather = pd.concat([weather,temp])

#clean column headers
weather = weather.rename(columns = {'Date':"Registration Date", 'Location': 'event_location','Mean TemperatureF':"Registration Day TemperatureF",'Prev_30d_Mean_Temp':'Registration Day Prev_30d_Mean_Temp'})
weather = weather.rename(columns=lambda x: x.strip())
weather = weather[["Registration Date",'event_location', 'Registration Day TemperatureF', 'Registration Day Prev_30d_Mean_Temp']]
#get weather of date registered
# data["Registration Date"] = data["event_date"] - pd.to_timedelta(data["days_out_registered"],unit='d')
data = pd.merge(data, weather, how='left')

#clean column headers
weather = weather.rename(columns = {'Registration Date':'event_date', "Registration Day TemperatureF": 'Mean TemperatureF','Registration Day Prev_30d_Mean_Temp':'Prev_30d_Mean_Temp'})
weather = weather.rename(columns=lambda x: x.strip())
weather = weather[["event_date",'event_location', 'Mean TemperatureF', 'Prev_30d_Mean_Temp']]
#merge to take only relevant dates
weather = pd.merge(data[["Event","event_date",'event_location']].copy(), weather, how='left', on=["event_date",'event_location']).drop_duplicates().sort_values(by=["Event","event_location","event_date"])

# g = weather[['Event','event_location',"event_date", 'Mean TemperatureF', 'Prev_30d_Mean_Temp']].drop_duplicates().sort_values(by=["Event","event_location","event_date"])
g = weather.copy().groupby(["Event","event_location"])
weather['Previous Year Mean TemperatureF'] = g['Mean TemperatureF'].apply(lambda x: x.shift())
weather['Previous Year 30d_Mean_Temp'] = g['Prev_30d_Mean_Temp'].apply(lambda x: x.shift())

weather = weather.reset_index()
data = pd.merge(data, weather, how='left')
data = data.drop("index",1)

# data["Registration Day/30d Temperature"] = data["Registration Day TemperatureF"] / data['Registration Day Prev_30d_Mean_Temp']
# data['Event Mean Temperature/Previous Year'] = data["Mean TemperatureF"] / data["Previous Year Mean TemperatureF"]
# data['Event Rolling 30d_Mean_Temp/Previous Year'] = data["Prev_30d_Mean_Temp"] / data["Previous Year 30d_Mean_Temp"]


Chicago
Atlanta
Columbus
Dallas
Denver
HighlandPark
LakeGeneva
Minneapolis
Nashville
Philadelphia
Phoenix
SanDiego
SanFrancisco
Seattle
St.Louis


In [13]:
expand_columns(data)
data.head()
# # data.columns
# # data[data["Registration Day/30d Temperature"] > 3]
# data.columns

,User ID,User Email,User First Name,User Last Name,Registrant ID,Registrant Email,Count,Registrant First Name,Registrant Last Name,Registrant Gender,Registrant Birthdate,Registrant Zipcode,Registration ID,Registration Event Name,Registration Category,distance_qty,distance_unit,event_location,event_date,fees_event,fundraising,fundraising_goal,days_out_registered,timestamp_registered,result_time,results_net_pace,fees_first,fees_first_until,fees_second,fees_second_until,fees_third,fees_third_until,fees_fourth,fees_fourth_until,Gender,Registration Date,Year,result_time_Minutes,results_net_pace_Minutes,Age,Age at Event,Kids,Category,Event,city_coord,zip_coord,zip_coord_latitude,zip_coord_longitude,Distance_To_Event(miles),Registration Day TemperatureF,Registration Day Prev_30d_Mean_Temp,Mean TemperatureF,Prev_30d_Mean_Temp,Previous Year Mean TemperatureF,Previous Year 30d_Mean_Temp
0,4d8aaf97-16e4-4b23-b421-0c4f7f000001,deannagill@comcast.net,Dede,Gill,4d8aaf97-9098-4707-b672-0c4f7f000001,NaN,8,Dede,Gill,Female,1975-01-19,60050,4d8aaf97-f208-42f8-9300-0c4f7f000001,2011 Hot Chocolate 15K/5K - Chicago,15K,15,kilometers,"Chicago, IL",2011-11-05,65,0,0,249,2011-02-28 11:20:07,NaN,NaN,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,2011-02-28,2011,NaN,NaN,41,36,NaN,15K,HOT CHOCOLATE,"(41.8119, -87.6873)","(42.3311, -88.2955)",42.3311,-88.2955,47.563132,32,27.733333,47,53.9,36,53.8
1,4d8aaf9b-9f08-4e10-94c9-0c4f7f000001,rswee@comcast.net,Robert,Swee,4d8aaf9b-2668-463b-aeb8-0c4f7f000001,NaN,15,Robert,Swee,Male,1954-03-27,60044,4d8aaf9b-bec0-46c8-b0a5-0c4f7f000001,2011 Hot Chocolate 15K/5K - Chicago,15K,15,kilometers,"Chicago, IL",2011-11-05,65,0,0,263,2011-02-14 20:50:30,01:11:03,00:07:38,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,2011-02-14,2011,71.050000,7.633333,62,57,NaN,15K,HOT CHOCOLATE,"(41.8119, -87.6873)","(42.282, -87.856)",42.2820,-87.8560,33.586086,37,21.166667,47,53.9,36,53.8
2,4d8aaf9e-ee50-47bf-87ad-0c4f7f000001,tom@gamainc.com,Tom,Golbeck,4d8aaf9e-5a10-4ff6-ba0b-0c4f7f000001,NaN,2,Tom,Golbeck,Male,1979-04-03,60012,4d8aaf9e-1a28-4865-ab37-0c4f7f000001,2011 Hot Chocolate 15K/5K - Chicago,15K,15,kilometers,"Chicago, IL",2011-11-05,65,0,0,263,2011-02-14 09:54:49,01:18:42,00:08:27,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,2011-02-14,2011,78.700000,8.450000,37,32,NaN,15K,HOT CHOCOLATE,"(41.8119, -87.6873)","(42.2662, -88.3213)",42.2662,-88.3213,45.245022,37,21.166667,47,53.9,36,53.8
3,4d8aafa9-11a0-4b2a-8f1d-0c4f7f000001,jorays@comcast.net,Rebecca,Joray,4d8aafa9-adb4-48db-af1a-0c4f7f000001,NaN,5,Rebecca,Joray,Female,1971-01-15,60046,4d8aafa9-5ea0-4ceb-ab09-0c4f7f000001,2011 Hot Chocolate 15K/5K - Chicago,15K,15,kilometers,"Chicago, IL",2011-11-05,65,0,0,263,2011-02-14 14:06:32,01:22:40,00:08:53,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Female,2011-02-14,2011,82.666667,8.883333,45,40,NaN,15K,HOT CHOCOLATE,"(41.8119, -87.6873)","(42.3813, -87.9991)",42.3813,-87.9991,42.441880,37,21.166667,47,53.9,36,53.8
4,4d8aafa9-d200-4997-8386-0c4f7f000001,JAMESREYNOLDSC@AOL.COM,James,Reynolds,4d8aafa9-7d50-4698-ace2-0c4f7f000001,NaN,23,James,Reynolds,Male,1945-09-03,60618,4d8aafaa-1a58-4726-9592-0c4f7f000001,2011 Hot Chocolate 15K/5K - Chicago,15K,15,kilometers,"Chicago, IL",2011-11-05,65,0,0,246,2011-03-03 11:25:01,01:41:43,00:10:55,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Male,2011-03-03,2011,101.716667,10.916667,70,66,NaN,15K,HOT CHOCOLATE,"(41.8119, -87.6873)","(41.9464, -87.7042)",41.9464,-87.7042,9.323526,36,28.633333,47,53.9,36,53.8


In [14]:

data.to_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\2016-06-07 registrant_data_Processed.csv", index=False)
#########Finished preprocessing transaction data

In [ ]:
#number of duplicates #some duplicates have one variable different such as net pace NaN
expand_columns(data)
# data.describe(include="all")
print(data.duplicated(subset=["Original Registrant ID", "Registration Event Name", "Registration Category"], keep=False).sum())
print(data.duplicated(subset=["Original Registrant ID", "Registration Event Name", "Registration Category"]).sum())

print(data.duplicated(subset=["Registrant ID", "Registration Event Name", "Registration Category"], keep=False).sum())
print(data.duplicated(subset=["Registrant ID", "Registration Event Name", "Registration Category"]).sum())

print(data.duplicated(subset=["Original Registrant ID","Registrant ID", "Registration Event Name", "Registration Category"], keep=False).sum())
print(data.duplicated(subset=["Original Registrant ID","Registrant ID", "Registration Event Name", "Registration Category"]).sum())

data[data.duplicated(subset=["Original Registrant ID","Registrant ID", "Registration Event Name", "Registration Category"], keep=False)].sort_values("Original Registrant ID")

In [ ]:
expand_columns(data)
data[data["fees_event"]<1]["Category"].unique()

In [ ]:
data[(data["fees_event"]<1) & (data["Category"]=="15K")]

In [ ]:
#some registrants have a total negative fees
data[data["Original Registrant ID"] == "4d8ab892-a1e0-440b-a353-0c4f7f000001"]

In [15]:
#########################################
# Begin Creation of analytics dataframe #
#########################################
import pandas as pd
import numpy as np
import matplotlib
import datetime as DT
def lastValue(x):
    #lambda x: x.dropna().iloc[-1]
    try:
        return x.dropna().iloc[-1]
    except:
        return x.iloc[-1]
def expand_columns(data):  pd.set_option('display.max_columns',len(data.columns))
def describe2(data):
    print("COUNT", "UNIQUE", "#_NULL","DATA_TYPE","COLUMN_NAME : FIRST ITEM", sep="\t")
    for column in data:
        print(data[column].count(),data[column].nunique(),data[column].isnull().sum(), str(data[column].dtypes)+"\t",str(column)+" :",data[column].iloc[0],sep="\t")
data = pd.read_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\2016-06-07 registrant_data_Processed.csv", low_memory=False,encoding = "ISO-8859-1")
data["Registrant Birthdate"] = pd.to_datetime(data["Registrant Birthdate"],format='%Y-%m-%d')#,errors='coerce') 
data["event_date"] = pd.to_datetime(data["event_date"],format='%Y-%m-%d')#,errors='coerce')#.dt.date

In [ ]:
# data = pd.read_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\registrant_data\registrant_data.csv", low_memory=False,encoding = "ISO-8859-1")
# data.head()

In [ ]:
# data["Event"] = data["Registration Event Name"].str.upper().str.extract('(HOT CHOCOLATE)')
# data["Year"] = pd.to_datetime(data["event_date"]).dt.year
# data["Registrant ID"] = data["Registrant ID"].fillna("FILL_NULL_w_USER_ID:"+data["User ID"])
# HC_2009 = data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2009) & (data["event_location"]=="Chicago, IL")]["Registrant ID"].tolist()
# HC_2010 = data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2010)& (data["event_location"]=="Chicago, IL")]["Registrant ID"].tolist()
# HC_2011 = data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2011)& (data["event_location"]=="Chicago, IL")]["Registrant ID"].tolist()
# HC_2012 = data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2012)& (data["event_location"]=="Chicago, IL")]["Registrant ID"].tolist()
# HC_2013 = data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2013)& (data["event_location"]=="Chicago, IL")]["Registrant ID"].tolist()
# HC_2014 = data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2014)& (data["event_location"]=="Chicago, IL")]["Registrant ID"].tolist()
# HC_2015 = data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2015)& (data["event_location"]=="Chicago, IL")]["Registrant ID"].tolist()


In [ ]:
# temp1 = ['One', 'Two', 'Three', 'Four']
# temp2 = ['One', 'Two','five']
# temp3 = list(set(temp1) - set(temp2))
# temp3

# temp15 = list(set(HC_2015) - set(HC_2013)-set(HC_2012) - set(HC_2011)-set(HC_2010) - set(HC_2009))
# temp14 = list(set(HC_2014) - set(HC_2013)-set(HC_2012) - set(HC_2011)-set(HC_2010) - set(HC_2009))
# data[(data["Event"]=="HOT CHOCOLATE") & (data["Year"]==2014)]["Registrant ID"].isin(temp15).sum()
# list(set(HC_2014) - set(temp15))

In [ ]:
# data["Event"] = data["Registration Event Name"].str.upper().str.extract('(HOT CHOCOLATE)')
# (data["Event"]=="HOT CHOCOLATE").sum()

In [ ]:
# data["Event"] = data["Registration Event Name"].str.upper().str.extract('(HOT CHOCOLATE)')
# data["Registration Event Name"]=data["Registration Event Name"].str.upper()
# data["?"] = data["Registration Event Name"].str.contains('HOT CHOCOLATE')

In [ ]:
# data["?"].sum()

In [16]:
#recreate GPS after loading processed data
import geopy
import geopy.distance
locations = pd.read_table("C:\\Users\\patrick.carey\\Downloads\\US.txt",encoding = "ISO-8859-1", low_memory=False, 
                          names=["country","zip","city","full_state","state","county", "n","n1","n2","latitude","longitude","n3"])
locations["zip"] = locations["zip"].astype(int).astype(str)
locations['GPS'] = tuple(zip(locations.latitude, locations.longitude))
locations['event_location'] = locations['city'] + ', ' + locations['state']
GPS_City = locations.set_index('event_location')['GPS'].to_dict()
GPS_Zip = locations.set_index('zip')['GPS'].to_dict()
GPS_Lat = locations.set_index('zip')['latitude'].to_dict()
GPS_Long = locations.set_index('zip')['longitude'].to_dict()
# data['city_coord'] = data['event_location'].map(GPS_City)
# data['zip_coord'] = data['Registrant Zipcode'].map(GPS_Zip)
# data['zip_coord_latitude'] = data['Registrant Zipcode'].map(GPS_Lat)
# data['zip_coord_longitude'] = data['Registrant Zipcode'].map(GPS_Long)
# data["Distance_To_Event(miles)"] = data.apply(lambda x: geopy.distance.distance(x['city_coord'], x['zip_coord']).miles, axis=1)

In [ ]:
data[data["Event"]=="HOT CHOCOLATE"]["event_location"].unique()

In [ ]:
expand_columns(data)
# data.head()
# data.describe()
# data.head()

In [ ]:
clist = ['Chicago, IL', 'Dallas, TX', 'San Diego, CA','Atlanta, GA', 'Seattle, WA', 'Columbus, OH',
         'Minneapolis, MN','Philadelphia, PA', 'San Francisco, CA', 'St. Louis, MO','Nashville, TN', 'Denver, CO']
# clist = ['Chicago, IL']
for c in clist: 
    try:
        #DataFrame to predict

        year = 2015 
        city = c
#         city = "Chicago, IL" 
        race = "HOT CHOCOLATE" 
        category = None
#         category = "15K"
#         category = "5K"


        regId_city = data[(data["event_location"]==city)]["Registrant ID"] #registration IDs that have been to an event in city

        # filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
        #        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
        #        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
        #        'KIDS RACE', '10 MILE']
        date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
        if category == None: 
            dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["Registrant ID"] 
            fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_" + city + "_" + str(year) + ".csv"
        else: 
            dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city) & (data["Category"]==category)]["Registrant ID"] 
            fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_" + city + "_" + str(year) + "_" + category +".csv"

        # # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
        # df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
        # df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

        #df of previous registrants that attended event in city
        df = data[(data["event_date"] < date) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
        print(df["Registrant ID"].nunique())
        df = df.drop("Count", 1)
        df["Age at Event"] = (pd.to_datetime(date) - df["Registrant Birthdate"]).astype('<m8[Y]')
        df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
        df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
        df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
        df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
        df['HOT CHOCOLATE'] = np.where(df['Event'].str.contains("HOT CHOCOLATE"), 1,0)
        dependent_variable = race + " " + str(year) + " " + city
        df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

        df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
        df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
        df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

        gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                                      'days_out_registered': np.mean,'results_net_pace_Minutes': np.mean, 'Distance_To_Event(miles)': np.mean, 
                                                      'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 
                                                      'Event Rolling 30d_Mean_Temp/Previous Year': np.mean,'Female': np.max, 'Male': np.max, '10K': np.sum,'15K': np.sum, 
                                                      '5 MILE': np.sum, '5K': np.sum, 'HOT CHOCOLATE': np.sum, dependent_variable: np.max})

        gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()
        def change_column_order(df, col_name, index):
            cols = df.columns.tolist()
            cols.remove(col_name)
            cols.insert(index, col_name)
            return df[cols]
#         gId = pd.concat([gId, gId2], axis=1)
        gId = gId.drop("Male", 1)
        gId = pd.DataFrame(gId)
        gId = gId.rename(columns = {'Year':'Year of Last Event'})
        gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
        gId = change_column_order(gId, dependent_variable, len(gId.columns))
        gId.columns = map(str.title, gId.columns)

        # zscore = lambda x: (x - x.mean()) / x.std()
        # g = gId[:-1].apply(zscore)
        print(len(gId))
        gId.to_csv(fileName)
    except:
        print("pass " + city)
        pass

In [ ]:
data.columns

In [ ]:
# clist = ['Chicago, IL', 'Dallas, TX', 'San Diego, CA','Atlanta, GA', 'Seattle, WA', 'Columbus, OH',
#          'Minneapolis, MN','Philadelphia, PA', 'San Francisco, CA', 'St. Louis, MO','Nashville, TN', 'Denver, CO']
clist = ['Chicago, IL']

#DataFrame to predict

year = 2015 
# city = c
city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


regId_city = data[(data["event_location"]==city)]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_" + city + "_" + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city) & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_" + city + "_" + str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["event_date"] < date) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(date) - df["Registrant Birthdate"]).astype('<m8[Y]')
df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year) + " " + city
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': np.mean, 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 
                                              'Event Rolling 30d_Mean_Temp/Previous Year': np.mean,'Female': np.max, 'Male': np.max, '10K': np.sum,'15K': np.sum, 
                                              '5 MILE': np.sum, '5K': np.sum, dependent_variable: np.max})

gId23 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()
gId2 = gId23[["KIDS","HOT CHOCOLATE"]].copy()
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)

# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
# gId.to_csv(fileName)
   

In [ ]:
data.columns
data["Category"].unique()

In [ ]:
expand_columns(data)
data.tail()

In [19]:
#1
#DataFrame to registrant ID
date = pd.to_datetime('3/11/2016') #date recieved files
df = data[(data["event_date"] < date)].copy().sort_values(by="event_date")
# df = data.copy().sort_values(by="event_date")
print(data["Registrant ID"].nunique())
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)

df = pd.get_dummies(df, columns=["Gender"],dummy_na=False, prefix="",prefix_sep="")

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Registrant Zipcode","distance_qty",'distance_unit'],1)

# gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.mean,"Age": np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
#                                               'days_out_registered': np.mean,'results_net_pace_Minutes': lambda x: lastValue(x), 'Distance_To_Event(miles)': np.mean, 
#                                               'Registration Day TemperatureF': lambda x: lastValue(x),'Registration Day Prev_30d_Mean_Temp': lambda x: lastValue(x), 
#                                               'Mean TemperatureF': lambda x: lastValue(x),'Prev_30d_Mean_Temp': lambda x: lastValue(x), 
#                                               'Previous Year Mean TemperatureF': lambda x: lastValue(x),'Previous Year 30d_Mean_Temp': lambda x: lastValue(x),
#                                               'Female': np.max, 'Male': np.max, '10K': np.max,'15K': np.max, '5 MILE': np.max, '5K': np.max, 
#                                               "zip_coord_latitude": np.mean, "zip_coord_longitude": np.mean, dependent_variable: np.max})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean
# , '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum

gId = df.copy().groupby("Registrant ID").mean()
gId2 = pd.get_dummies(data.copy()[["Registrant ID","Registration Category","Category","Kids","Event"]], columns = ["Category","Kids","Event"],dummy_na=False, prefix="",prefix_sep="")
gId2['5 MILE'] = np.where(gId2['Registration Category'].str.contains("TRIPLE NICKEL"), 1,gId2['5 MILE']) 
gId2['5K'] = np.where(gId2['Registration Category'].str.contains("TRIPLE NICKEL"), 1,gId2['5K'])
gId2 = gId2.groupby("Registrant ID").sum()
gId2 = gId2.drop("TRAINING",1)

gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
# gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)

gId.columns = map(str.title, gId.columns)
#balance data
# gId = gId.dropna()
# y = gId[gId[dependent_variable.title()] ==1].copy()
# n = gId[gId[dependent_variable.title()] ==0].copy().sample(n=(len(y.index)))
# yn = pd.concat([y, n], axis=0)
# yn.to_csv(fileName)
# print(len(yn))
# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)

print(len(gId))
gId.to_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\Ram_Registrants.csv")
#fill na with mean
gId= gId.fillna(gId.mean())
gId.to_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\Ram_RegistrantsMean.csv")

671414
649407
671414


In [20]:
expand_columns(gId)
gId.head()

,10 Mile,10K,15K,5 Mile,5K,Age,Age At Event,Bangs Lake Multisport Festival,Big Foot Triathlon,Btn Big 10K,Bucktown 5K,Cinco De Miler,Concert,Days_Out_Registered,Distance_To_Event(Miles),Donation,Fees_Event,Female,Fleet Feet Sports Soldier Field 10 Mile,Fundraising,Fundraising_Goal,Goodie Bag,Half Marathon,Hot Chocolate,Humboldt Park 5K,Kids,Kids Race,Male,Mean Temperaturef,Miscellaneous,North Shore Half Marathon,North Shore Turkey Trot,Prev_30D_Mean_Temp,Previous Year 30D_Mean_Temp,Previous Year Mean Temperaturef,Raffle,Registration Day Prev_30D_Mean_Temp,Registration Day Temperaturef,Result_Time_Minutes,Results_Net_Pace_Minutes,Rock The Night 5K,Season Pass,South Shore Triathlon,Terrapin,Tri/Other,Triple Nickel,Valet/Parking,Walk,Walk For Little City,Year,Zip_Coord_Latitude,Zip_Coord_Longitude
4d8aaf92-1770-468a-8c81-0c4f7f000001,0,0,1,0,1,57,50.666667,1,0,0,0,0,0,30.000000,51.964078,0,61.333333,0,0,0,0,0,0,1,0,0,0,1,55.500000,0,0,0,65.916667,60.250000,57.000000,0,75.616667,80.500000,52.083333,8.333333,0,0,0,1,1,0,0,0,0,2009.666667,41.4706,-87.0783
4d8aaf92-17a0-42f6-9855-0c4f7f000001,0,0,0,0,0,54,47.000000,0,1,0,0,0,0,3.000000,36.025178,0,89.000000,0,0,0,0,0,0,0,0,0,0,1,74.000000,0,0,0,67.833333,51.052010,48.104333,0,65.666667,84.000000,63.283519,11.718620,0,0,0,0,1,0,0,0,0,2009.000000,43.1095,-88.4862
4d8aaf92-2430-4b7e-ab0d-0c4f7f000001,0,0,0,0,0,40,34.000000,0,3,0,0,0,0,96.333333,59.379769,0,99.000000,0,0,0,0,0,0,0,0,0,0,1,69.666667,0,0,0,66.522222,67.466667,73.500000,0,48.716667,48.500000,63.283519,11.718620,0,0,0,0,3,0,0,0,0,2010.000000,43.0696,-89.4239
4d8aaf92-2e90-4fc1-8bef-0c4f7f000001,0,0,0,0,0,55,48.500000,0,2,0,0,0,0,12.000000,36.025178,0,94.000000,0,0,0,0,0,0,0,0,0,0,1,73.500000,0,0,0,67.466667,67.833333,74.000000,0,63.483333,64.000000,63.283519,11.718620,0,0,0,0,2,0,0,0,0,2009.500000,43.1095,-88.4862
4d8aaf92-37c0-40cf-b40a-0c4f7f000001,0,0,0,0,0,48,41.500000,2,0,0,0,0,0,2.000000,47.366448,0,99.000000,0,0,0,0,0,0,0,0,0,0,1,48.993549,0,0,0,51.736506,51.052010,48.104333,0,58.935234,56.469029,63.283519,11.718620,0,0,0,0,2,0,0,0,0,2009.500000,41.6018,-87.8899


In [ ]:
#7
#DataFrame to predict all HC

year = 2015 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"].dropna() #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': lambda x: lastValue(x), 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day TemperatureF': lambda x: lastValue(x),'Registration Day Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Mean TemperatureF': lambda x: lastValue(x),'Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Previous Year Mean TemperatureF': lambda x: lastValue(x),'Previous Year 30d_Mean_Temp': lambda x: lastValue(x),
                                              'Female': np.max, 'Male': np.max, '10K': np.max,'15K': np.max, '5 MILE': np.max, '5K': np.max, 
                                              "zip_coord_latitude": np.mean, "zip_coord_longitude": np.mean, dependent_variable: np.max})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean
# , '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum

gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event","Year"]].copy(), columns = ['Year',"Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").max()

def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
# gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)
#balance data
# gId = gId.dropna()
# y = gId[gId[dependent_variable.title()] ==1].copy()
# n = gId[gId[dependent_variable.title()] ==0].copy().sample(n=(len(y.index)))
# yn = pd.concat([y, n], axis=0)
# yn.to_csv(fileName)
# print(len(yn))
# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
#6
#DataFrame to predict all HC

year = 2015 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
# category = None
category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"] #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
# df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({dependent_variable: np.max,"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': lambda x: lastValue(x), 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day TemperatureF': lambda x: lastValue(x),'Registration Day Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Mean TemperatureF': lambda x: lastValue(x),'Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Previous Year Mean TemperatureF': lambda x: lastValue(x),'Previous Year 30d_Mean_Temp': lambda x: lastValue(x),'Male': np.max,
                                              "distance_qty": np.mean})
#                                               'Female': np.max, , '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean
# , '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum

# gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()
gId2 = pd.get_dummies(df[["Registrant ID","Year"]].copy(), columns = ["Year"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").max()
# gId2 = gId23[["KIDS","HOT CHOCOLATE"]].copy()
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
# gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)

# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
#5
#DataFrame to predict all HC

year = 2015 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"].dropna() #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({dependent_variable: np.max,"Age at Event": np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': lambda x: lastValue(x), 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day TemperatureF': lambda x: lastValue(x),'Registration Day Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Mean TemperatureF': lambda x: lastValue(x),'Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Previous Year Mean TemperatureF': lambda x: lastValue(x),'Previous Year 30d_Mean_Temp': lambda x: lastValue(x),'Male': np.max})#,
#                                               'Female': np.max, '10K': np.max,'15K': np.max, '5 MILE': np.max, '5K': np.max})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean
# , '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum

# gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event","Year"]].copy(), columns = ['Year',"Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").max()

def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
# gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
# gId = gId.drop("TRAINING", 1)
# gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)
#balance data
# gId = gId.dropna()
# y = gId[gId[dependent_variable.title()] ==1].copy()
# n = gId[gId[dependent_variable.title()] ==0].copy().sample(n=(len(y.index)))
# yn = pd.concat([y, n], axis=0)
# yn.to_csv(fileName)
# print(len(yn))
# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
#4
#DataFrame to predict all HC

year = 2015 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"].dropna() #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': lambda x: lastValue(x), 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day TemperatureF': lambda x: lastValue(x),'Registration Day Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Mean TemperatureF': lambda x: lastValue(x),'Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Previous Year Mean TemperatureF': lambda x: lastValue(x),'Previous Year 30d_Mean_Temp': lambda x: lastValue(x),
                                              'Female': np.max, 'Male': np.max, '10K': np.max,'15K': np.max, '5 MILE': np.max, '5K': np.max, dependent_variable: np.max})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean
# , '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum

gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event","Year"]].copy(), columns = ['Year',"Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").max()

def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
# gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)
#balance data
# gId = gId.dropna()
# y = gId[gId[dependent_variable.title()] ==1].copy()
# n = gId[gId[dependent_variable.title()] ==0].copy().sample(n=(len(y.index)))
# yn = pd.concat([y, n], axis=0)
yn.to_csv(fileName)
print(len(yn))
# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
#3
#DataFrame to predict all HC

year = 2015 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"] #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': lambda x: lastValue(x), 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day TemperatureF': lambda x: lastValue(x),'Registration Day Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Mean TemperatureF': lambda x: lastValue(x),'Prev_30d_Mean_Temp': lambda x: lastValue(x), 
                                              'Previous Year Mean TemperatureF': lambda x: lastValue(x),'Previous Year 30d_Mean_Temp': lambda x: lastValue(x),
                                              'Female': np.max, 'Male': np.max, '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum, dependent_variable: np.max})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean
# , '10K': np.sum,'15K': np.sum, '5 MILE': np.sum, '5K': np.sum

gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()
# gId2 = gId23[["KIDS","HOT CHOCOLATE"]].copy()
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
# gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)

# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
#2
#DataFrame to predict all HC

year = 2015 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"] #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': np.mean, 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day TemperatureF': np.mean,'Registration Day Prev_30d_Mean_Temp': np.mean, 'Mean TemperatureF': np.mean,
                                              'Prev_30d_Mean_Temp': np.mean, 'Previous Year Mean TemperatureF': np.mean,'Previous Year 30d_Mean_Temp': np.mean,
                                              'Female': np.max, 'Male': np.max, '10K': np.sum,'15K': np.sum, 
                                              '5 MILE': np.sum, '5K': np.sum, dependent_variable: np.max})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean

gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()
# gId2 = gId23[["KIDS","HOT CHOCOLATE"]].copy()
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)

# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
#1
#DataFrame to predict all HC

year = 201 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"] #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': np.mean, 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 
                                              'Event Rolling 30d_Mean_Temp/Previous Year': np.mean,'Female': np.max, 'Male': np.max, '10K': np.sum,'15K': np.sum, 
                                              '5 MILE': np.sum, '5K': np.sum, dependent_variable: np.max})

gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()
# gId2 = gId23[["KIDS","HOT CHOCOLATE"]].copy()
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)

# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
# gId.to_csv(fileName)
expand_columns(gId)
gId.head()
gId.describe()

In [ ]:
gId.columns

In [ ]:
gId2.head()

In [ ]:
expand_columns(gId)
gId.head()
gId.describe()

In [ ]:

#DataFrame to predict all HC

year = 2015 
# city = c
# city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None
#         category = "15K"
#         category = "5K"


cities = data[(data["Event"]==race) & (data["Year"]==year)]["event_location"] #cities with HC in Year
regId_city = data[(data["event_location"].isin(cities))]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
# date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_"   + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year)  & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_"  +  str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["Year"] < year) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df["Age at Event"] = (pd.to_datetime(year, format="%Y") - df["Registrant Birthdate"]).astype('<m8[Y]')
# df["Distance_To_Event(miles)"] = df.apply(lambda x: geopy.distance.distance(GPS_City[city], x['zip_coord']).miles, axis=1)
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])
dependent_variable = race + " " + str(year)
df[dependent_variable] = np.where(df["Registrant ID"].isin(dv_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': np.mean, 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day TemperatureF': np.mean,'Registration Day Prev_30d_Mean_Temp': np.mean, 'Mean TemperatureF': np.mean,
                                              'Prev_30d_Mean_Temp': np.mean, 'Previous Year Mean TemperatureF': np.mean,'Previous Year 30d_Mean_Temp': np.mean,
                                              'Female': np.max, 'Male': np.max, '10K': np.sum,'15K': np.sum, 
                                              '5 MILE': np.sum, '5K': np.sum, dependent_variable: np.max})
#'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 'Event Rolling 30d_Mean_Temp/Previous Year': np.mean

gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()
# gId2 = gId23[["KIDS","HOT CHOCOLATE"]].copy()
def change_column_order(df, col_name, index):
    cols = df.columns.tolist()
    cols.remove(col_name)
    cols.insert(index, col_name)
    return df[cols]
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
gId = gId.rename(columns = {'Year':'Year of Last Event'})
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)
gId = change_column_order(gId, dependent_variable, len(gId.columns))
gId.columns = map(str.title, gId.columns)

# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(fileName)
   

In [ ]:
#DataFrame to registrant ID
date = pd.to_datetime('3/11/2016') #date recieved files
df = data[(data["event_date"] < date)].copy().sort_values(by="event_date")
# df = data.copy().sort_values(by="event_date")
print(data["Registrant ID"].nunique())
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)

df = pd.get_dummies(df, columns=["Gender"],dummy_na=False, prefix="",prefix_sep="")

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max,"Age": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': np.mean, 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 
                                              'Event Rolling 30d_Mean_Temp/Previous Year': np.mean,'Female': np.max, 'Male': np.max})

gId2 = pd.get_dummies(data.copy()[["Registrant ID","Registration Category","Category","Kids","Event"]], columns = ["Category","Kids","Event"],dummy_na=False, prefix="",prefix_sep="")
gId2['5 MILE'] = np.where(gId2['Registration Category'].str.contains("TRIPLE NICKEL"), 1,gId2['5 MILE']) 
gId2['5K'] = np.where(gId2['Registration Category'].str.contains("TRIPLE NICKEL"), 1,gId2['5K'])
gId2 = gId2.groupby("Registrant ID").sum()
gId2 = gId2.drop("TRAINING",1)
gId = pd.concat([gId, gId2], axis=1)
gId = pd.DataFrame(gId)
gId = gId.rename(columns = {'Year':'Year of Last Event',"Age at Event":"Age at Last Event" })
gId = gId.reindex_axis(sorted(gId.columns, key=str.lower), axis=1)

gId.columns = map(str.title, gId.columns)

# zscore = lambda x: (x - x.mean()) / x.std()
# g = gId[:-1].apply(zscore)
print(len(gId))
gId.to_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\Ram_Registrants.csv")
#fill na with mean
gId= gId.fillna(gId.mean())
gId.to_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\Ram_RegistrantsMean.csv")

In [ ]:
d = pd.read_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\Ram_Registrants.csv", low_memory=False)

In [ ]:
d.head()

In [ ]:
d["Unnamed: 0"].nunique()

In [ ]:
gId.columns.duplicated()

In [ ]:
gId.columns

In [ ]:
# data.columns
# data[['Original Registrant ID','Registrant ID']]

In [ ]:
gId2

In [ ]:
df = df.drop("index",1)
df = df.drop(["Mean TemperatureF_y","Prev_30d_Mean_Temp_y","Previous Year Mean TemperatureF_y","Previous Year 30d_Mean_Temp_y"],1)
# dependent_variable = race + " " + str(year) + " " + city

#fill na with mean
# df = df.fillna(df.mean())
# df["Dependent Variable"] = np.where(df["Registrant ID"].isin(dv_list), 1,0)
print(df["Registrant ID"].nunique())
df.to_csv(fileName, index=False)

In [ ]:
expand_columns(df)
df.head()

In [ ]:
describe2(df)

In [ ]:
expand_columns(df)
df.head()

In [ ]:

#DF to predict 2015 Chicago Hot Chocolate 
year = 2015 
city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None

regId_city = data[(data["event_location"]==city)]["Registrant ID"] #registration IDs that have been to an event in city
relevantList = ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE','WALK', 'TRIPLE NICKEL', 'KIDS RACE', '10 MILE'] 
# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_" + city + "_" + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city) & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_" + city + "_" + str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["event_date"] < date) &(data["Registrant ID"].isin(regId_city))].copy() 
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df = pd.get_dummies(df, columns=["Gender"],dummy_na=False, prefix="",prefix_sep="")

# #grouped data
# g = df[["Registration Category","Registrant ID","Category","Event","Kids"]].copy()  
# #dummy variable creation
# #get dummies will remove original, so create a copy first
# g = pd.get_dummies(g, columns=["Category","Event","Kids"],dummy_na=False, prefix="",prefix_sep="")
# #add 5 mile and 5k to triple nickel 
# if "TRIPLE NICKEL" in g.columns:
#     g['5 MILE'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5 MILE']) 
#     g['5K'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5K']) 
# g = g.drop("Registration Category", axis=1)
# g = g.groupby("Registrant ID", as_index=False).sum() #group and sum



# print(df["Registrant ID"].nunique())
# df = df[df["Category"].isin(relevantList)] ######### retrieve relevant race data
# print(df["Registrant ID"].nunique())
# df = df.sort_values(by="event_date").drop_duplicates(subset="Registrant ID", keep="last") ########### keep last race
# df = pd.merge(df, g, how='left',on="Registrant ID") 
# df['event_date_y'] = date

# df = pd.merge(df, df_y, how='left',on="event_date_y")
# df["last_event_days_back"] = (df["event_date_y"] - df["event_date"])/ np.timedelta64(1, 'D') 
# #replace distance to event with distance to event y
# df = df.drop(["Distance_To_Event(miles)"],1)
# df['city_coord_y'] = df['event_location_y'].map(GPS_City)
# df["Distance_To_Event(miles)_y"] = df.apply(lambda x: geopy.distance.distance(x['city_coord_y'], x['zip_coord']).miles, axis=1)


# df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1) 
# df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
# df = df.drop(["Age","Registrant Zipcode","distance_qty"],1)

# df = df.drop("index",1)
# df = df.drop(["Mean TemperatureF_y","Prev_30d_Mean_Temp_y","Previous Year Mean TemperatureF_y","Previous Year 30d_Mean_Temp_y"],1)
# # dependent_variable = race + " " + str(year) + " " + city

# #fill na with mean
# # df = df.fillna(df.mean())
# #groupby
# #  f = lambda x: x.fillna(x.mean())
# # ransformed = grouped.transform(f)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

# # df["Dependent Variable"] = np.where(df["Registrant ID"].isin(dv_list), 1,0)
# print(df["Registrant ID"].nunique())
# df.to_csv(fileName, index=False)

In [ ]:
describe2(df)

In [ ]:

#DF to predict 2015 Chicago Hot Chocolate 
year = 2015 
city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None


regId_city = data[(data["event_location"]==city)]["Registrant ID"] #registration IDs that have been to an event in city

# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/" + race + "_" + city + "_" + str(year) + ".csv"
else: 
    dv_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city) & (data["Category"]==category)]["Registrant ID"] 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/"+race + "_" + city + "_" + str(year) + "_" + category +".csv"

# # df_y includes event info, use known weather  # for future we can use average of event or some other predicted weather
# df_y = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
# df_y.columns = [str(col) + '_y' for col in df_y.columns] #add suffix for weather of event being predicted

#df of previous registrants that attended event in city
df = data[(data["event_date"] < date) &(data["Registrant ID"].isin(regId_city))].copy().sort_values(by="event_date")
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df = df.rename(columns = {'Year':'Year of Last Event'}
df["Age at Event"] = (pd.to_datetime(date) - df["Registrant Birthdate"]).astype('<m8[Y]')
df = pd.get_dummies(df, columns=["Gender","Category"],dummy_na=False, prefix="",prefix_sep="")
df['5 MILE'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5 MILE']) 
df['5K'] = np.where(df['Registration Category'].str.contains("TRIPLE NICKEL"), 1,df['5K'])

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

gId = df.copy().groupby("Registrant ID").agg({"Age at Event": np.max, 'Year': np.max,'fees_event': np.mean, 'fundraising': np.mean, 'fundraising_goal': np.mean,
                                              'days_out_registered': np.mean,'results_net_pace_Minutes': np.mean, 'Distance_To_Event(miles)': np.mean, 
                                              'Registration Day/30d Temperature': np.mean,'Event Mean Temperature/Previous Year': np.mean, 
                                              'Event Rolling 30d_Mean_Temp/Previous Year': np.mean,'Female': np.max, 'Male': np.max, '10K': np.sum,'15K': np.sum, 
                                              '5 MILE': np.sum, '5K': np.sum})

gId2 = pd.get_dummies(df[["Registrant ID","Kids","Event"]].copy(), columns = ["Kids", "Event"],dummy_na=False, prefix="",prefix_sep="").groupby("Registrant ID").sum()

gId = pd.concat([gId, gId2], axis=1)
gId["Dependent Variable"] = np.where(gId["Registrant ID"].isin(dv_list), 1,0)               
#                                        , 'CONCERT': np.sum, 'DONATION': np.sum, 'HALF MARATHON': np.sum,'KIDS RACE': np.sum, 
#                                        'RAFFLE': np.sum, 'SEASON PASS': np.sum, 'TRAINING': np.sum, 'TRI/OTHER': np.sum, 'VALET/PARKING': np.sum, 
#                                        'WALK': np.sum,'BANGS LAKE MULTISPORT FESTIVAL': np.sum, 'BIG FOOT TRIATHLON': np.sum, 'BUCKTOWN 5K': np.sum,
#                                        'CINCO DE MILER': np.sum, 'HOT CHOCOLATE': np.sum, 'HUMBOLDT PARK 5K': np.sum, 'MISCELLANEOUS': np.sum,
#                                        'NORTH SHORE HALF MARATHON': np.sum, 'NORTH SHORE TURKEY TROT': np.sum, 'ROCK THE NIGHT 5K': np.sum, 
#                                        'SOUTH SHORE TRIATHLON': np.sum, 'TERRAPIN': np.sum, 'TRAINING': np.sum, 'WALK FOR LITTLE CITY': np.sum, 'KIDS': np.sum})

# relevantList = ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE','WALK', 'TRIPLE NICKEL', 'KIDS RACE', '10 MILE'] 

# 'Registration Day TemperatureF', 'Registration Day Prev_30d_Mean_Temp',
#        'Mean TemperatureF', 'Prev_30d_Mean_Temp',
#        'Previous Year Mean TemperatureF', 'Previous Year 30d_Mean_Temp',
#        'Registration Day/30d Temperature',
#        'Event Mean Temperature/Previous Year',
#        'Event Rolling 30d_Mean_Temp/Previous Year', 'Female', 'Male', '10K',
#        '15K', '5 MILE', '5K', 'CONCERT', 'DONATION', 'HALF MARATHON',
#        'KIDS RACE', 'RAFFLE', 'SEASON PASS', 'TRAINING', 'TRI/OTHER',
#        'TRIPLE NICKEL', 'VALET/PARKING', 'WALK',
#        'BANGS LAKE MULTISPORT FESTIVAL', 'BIG FOOT TRIATHLON', 'BUCKTOWN 5K',
#        'CINCO DE MILER', 'HOT CHOCOLATE', 'HUMBOLDT PARK 5K', 'MISCELLANEOUS',
#        'NORTH SHORE HALF MARATHON', 'NORTH SHORE TURKEY TROT',
#        'ROCK THE NIGHT 5K', 'SOUTH SHORE TRIATHLON', 'TERRAPIN', 'TRAINING',
#        'WALK FOR LITTLE CITY', 'KIDS'

# #grouped data
# g = df[["Registration Category","Registrant ID","Category","Event","Kids"]].copy()  
# #dummy variable creation
# #get dummies will remove original, so create a copy first
# g = pd.get_dummies(g, columns=["Category","Event","Kids"],dummy_na=False, prefix="",prefix_sep="")
# #add 5 mile and 5k to triple nickel 
# if "TRIPLE NICKEL" in g.columns:
#     g['5 MILE'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5 MILE']) 
#     g['5K'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5K']) 
# g = g.drop("Registration Category", axis=1)
# g = g.groupby("Registrant ID", as_index=False).sum() #group and sum



# print(df["Registrant ID"].nunique())
# df = df[df["Category"].isin(relevantList)] ######### retrieve relevant race data
# print(df["Registrant ID"].nunique())
# df = df.sort_values(by="event_date").drop_duplicates(subset="Registrant ID", keep="last") ########### keep last race
# df = pd.merge(df, g, how='left',on="Registrant ID") 
# df['event_date_y'] = date

# df = pd.merge(df, df_y, how='left',on="event_date_y")
# df["last_event_days_back"] = (df["event_date_y"] - df["event_date"])/ np.timedelta64(1, 'D') 
# #replace distance to event with distance to event y
# df = df.drop(["Distance_To_Event(miles)"],1)
# df['city_coord_y'] = df['event_location_y'].map(GPS_City)
# df["Distance_To_Event(miles)_y"] = df.apply(lambda x: geopy.distance.distance(x['city_coord_y'], x['zip_coord']).miles, axis=1)


# df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1) 
# df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
# df = df.drop(["Age","Registrant Zipcode","distance_qty"],1)

# df = df.drop("index",1)
# df = df.drop(["Mean TemperatureF_y","Prev_30d_Mean_Temp_y","Previous Year Mean TemperatureF_y","Previous Year 30d_Mean_Temp_y"],1)
# # dependent_variable = race + " " + str(year) + " " + city

# #fill na with mean
# # df = df.fillna(df.mean())
# #groupby
# #  f = lambda x: x.fillna(x.mean())
# # ransformed = grouped.transform(f)

# df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
# df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
# df = df.drop(["Age","Registrant Zipcode","distance_qty",'distance_unit'],1)

# df["Dependent Variable"] = np.where(df["Registrant ID"].isin(dv_list), 1,0)
# print(df["Registrant ID"].nunique())
# df.to_csv(fileName, index=False)


In [ ]:
# df.isnull().any()
df[df["Registrant ID"].isnull()]#.any(axis=1)]

In [ ]:

#DF to predict 2015 Chicago Hot Chocolate 
year = 2015 
city = "Chicago, IL" 
race = "HOT CHOCOLATE" 
category = None


relevantList = ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE','WALK', 'TRIPLE NICKEL', 'KIDS RACE', '10 MILE'] 
# filter out from ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE',
#        'TRAINING', 'DONATION', 'VALET/PARKING', 'WALK', 'RAFFLE',
#        'GOODIE BAG', 'CONCERT', 'TRIPLE NICKEL', 'SEASON PASS',
#        'KIDS RACE', '10 MILE']
date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique() [0]
if category == None: 
    id_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["Registrant ID"].to_dict() 
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/test/" + race + "_" + city + "_" + str(year) + ".csv"
else: 
    id_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city) & (data["Category"]==category)]["Registrant ID"].to_dict()
    fileName = "C:/Users/patrick.carey/Desktop/RamRacing/test/"+race + "_" + city + "_" + str(year) + "_" + category +".csv"


df = data[data["event_date"] < date].copy() 
print(df["Registrant ID"].nunique())
df = df.drop("Count", 1)
df = pd.merge(df,weather, how="left") #add weather for x
g = df[["Registrant ID", 'Registration Category',"Category","Event","Kids"]].copy() #df for later grouped data 

#dummy variable creation
#get dummies will remove original, so create a copy first
g["Event_"] = g["Event"]
g["Category_"] = g["Category"]
df["Gender"] = df["Registrant Gender"] 
df = pd.get_dummies(df, columns=["Gender"],dummy_na=False, prefix="",prefix_sep="")
g["Kids_"] = g["Kids"]
g = pd.get_dummies(g, columns=["Category","Event","Kids"],dummy_na=False, prefix="",prefix_sep="")
#add 5 mile and 5k to triple nickel 
g['5 MILE'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5 MILE']) 
g['5K'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5K']) 

g = g.drop('Registration Category',1) 
g = g.groupby("Registrant ID", as_index=False).sum() #group and sum
w = weather[(weather["event_location"] == city) & (weather["event_date"] == date)].copy() #weather for event
w.columns = [str(col) + '_y' for col in w.columns] #add suffix for weather ofevent being predicted

print(df["Registrant ID"].nunique())
df = df[df["Category"].isin(relevantList)] ######### retrieve relevant race data
print(df["Registrant ID"].nunique())
df = df.sort_values(by="event_date").drop_duplicates(subset="Registrant ID", keep="last") ########### keep last race
df = pd.merge(df, g, how='left',on="Registrant ID") 
df['event_date_y'] = date
df = pd.merge(df, w, how='left',on="event_date_y")
df["last_event_days_back"] = (df["event_date_y"] - df["event_date"])/ np.timedelta64(1, 'D') 

# dependent_variable = race + " " + str(year) + " " + city
df["Dependent Variable"] = np.where(df["Registrant ID"].isin(id_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1) 
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1) 
df = df.drop(["Age","Registrant Zipcode","distance_qty"],1)
# df = df.drop(["Mean TemperatureF_y","Prev_30d_Mean_Temp_y","Previous Year Mean TemperatureF_y","Previous Year 30d_Mean_Temp_y"],1)
print(df["Registrant ID"].nunique())
df.to_csv(fileName, index=False)

In [ ]:
data["Registrant ID"].nunique()

In [ ]:
expand_columns(df)
df.head()

In [ ]:
######create new df for queries
####drop NA for necessary columns #######
df =data.copy().dropna(subset=["User ID","Registrant Birthdate","Registrant ID", "Registrant Gender","Age","Age at Event"])
df["Year"] = df["Year"].astype(int)
#set new index
df = df.sort_values(by=['Year','Category', 'Event','event_location']) #sort for faster indexing
df["event_date"] = df["event_date"].dt.date #drop time piece
df.set_index(['Category', 'Event'], inplace=True)
df[df['Year']>2016].index.unique()



In [ ]:
import itertools
df = data.copy()
r = len(df["Category"].unique())
cs = []
for i in range(r-1):
    temp = tuple(itertools.combinations(df["Category"].unique(), i))
    for z in temp:
        cs.append(z)

print(len(cs))


In [ ]:
# expand_columns(g)
# g.head()

# expand_columns(w)
# w.head()

expand_columns(df)
df.describe(include="all")
df.tail()

In [ ]:
df["Category"].unique()

In [ ]:
expand_columns(df)
df[(df["Event"] == race)  & (df["event_location"]==city) & (df["Dependent Variable"]==1)].tail()

In [ ]:
#dummy variable creation
#get dummies will remove original, so create a copy first
data["Event_"] = data["Event"]
data["Category_"] = data["Category"]
data["Gender"] = data["Registrant Gender"]
data["Kids_"] = data["Kids"]
# data["Registration Event Name_"] = data["Registration Event Name"]
data = pd.get_dummies(data, columns=["Gender","Category_","Event_","Kids_"],dummy_na=False, prefix="",prefix_sep="")

#add 5 mile and 5k to triple nickel
data['5 MILE'] = np.where(data['Registration Category'].str.contains("TRIPLE NICKEL"), 1,data['5 MILE'])
data['5K'] = np.where(data['Registration Category'].str.contains("TRIPLE NICKEL"), 1,data['5K'])

#event date dummies
# data['event_location_'] = data['event_location']
# data = pd.get_dummies(data, columns=['event_location_'],dummy_na=False, prefix="",prefix_sep="_")

In [ ]:
expand_columns(data)
data.head()
# data[data["Category"] =="GOODIE BAG"].head()

In [ ]:
### create column for dependent variable, 2015 race attendence
dependent = data.copy()


In [ ]:
########### Retrieve locations
import pickle
import geopy
import geopy.distance

with open('GPS_Ram.pickle', 'rb') as handle: 
    GPS_Ram = pickle.load(handle) #load saved dict
# GPS_Ram = {} #only for creating new dict

geolocator = geopy.Nominatim()
for index, row in data.iterrows():
    eventCity = [index,row['event_location']][1]
    registrantZip = [index,row['Registrant Zipcode']][1]
    
    if eventCity == None:
        continue
    elif eventCity in GPS_Ram.keys():
        eventCityGPS = GPS_Ram[eventCity]
    else:
        try:
            eventCityRequest = geolocator.geocode(eventCity, timeout=10)
            eventCityGPS = (eventCityRequest.latitude, eventCityRequest.longitude)
            GPS_Ram[eventCity] = eventCityGPS
        except:
            continue
        
    if registrantZip == None:
        continue
    elif registrantZip in GPS_Ram.keys():
        registrantZipGPS = GPS_Ram[registrantZip]
    else:
        try:
            registrantZipRequest = geolocator.geocode(registrantZip, timeout=10)
            registrantZipGPS = (registrantZipRequest.latitude, registrantZipRequest.longitude)
            GPS_Ram[registrantZip] = registrantZipGPS
        except:
            continue

    data.loc[index,"Distance_To_Event"] = geopy.distance.distance(eventCityGPS, registrantZipGPS).miles
        

with open('GPS_Ram.pickle', 'wb') as handle:
    pickle.dump(GPS_Ram, handle)

In [ ]:
######create new df for queries
####drop NA for necessary columns #######
df =data.copy().dropna(subset=["User ID","Registrant Birthdate","Registrant ID", "Registrant Gender","Age","Age at Event"])
#set new index
df = df.sort_values(by=['Year','Category', 'Event','event_location']) #sort for faster indexing
df["event_date"] = df["event_date"].dt.date #drop time piece
df.set_index(['Year','Category', 'Event','event_location'], inplace=True)

In [ ]:
#split into pre and post 2015
df.loc["2015"].head()
df.loc["Year" < 2015].head()

In [ ]:
x = df.loc["15K", "HOT CHOCOLATE","Chicago, IL"].copy()
x = x.groupby('Registrant ID',as_index=False).sum()
x.to_csv(r"x.csv")
# x["Mean Temperature"] = x["event_date"].map(weatherDict['Chicago']["event_date"]["Mean TemperatureF"])

In [ ]:
x.head()

In [ ]:
d = df.loc["15K","HOT CHOCOLATE","Chicago, IL","2009":"2015"]
d.to_csv(r"C:\Users\patrick.carey\Desktop\RamRacing\x.csv")

In [ ]:
# expand_columns(df)
# df.loc[('15K','HOT CHOCOLATE'):("GOODIE BAG","HOT CHOCOLATE")].head() #[len("User ID") > 1].head()

In [ ]:
####drop NA for necessary columns #######
data =data.copy().dropna(subset=["User ID","Registrant Birthdate","Registrant ID", "Registrant Gender","Age","Age at Event"])
# describe2(data)

#####drop duplicate registrant entries #####
# data = data.drop_duplicates() # 921791 - 921776 = 15
# data = data.drop_duplicates(subset=["Registrant ID", "Registration Event Name","Registration Category","event_date"]) #duplicated regisrations by different Users,etc.
# describe2(data)

In [ ]:
data.event_location.unique()

In [ ]:
weatherDict["Chicago"][[0,2]].head()
weatherDict["Chicago"].insert(0, "Location", "Chicago, IL").head()

#DF to predict 2015 Chicago Hot Chocolate
year = 2015
city = "Chicago, IL"
race = "HOT CHOCOLATE"
category = "15K"


relevantList = ['15K', '5K', 'TRI/OTHER', 'HALF MARATHON', '10K', '5 MILE','WALK', 'TRIPLE NICKEL', 'KIDS RACE', '10 MILE'] #filter out goodie bags, concerts, etc.

date = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["event_date"].unique()
if category == None:
    id_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city)]["Registrant ID"]
    fileName = "C:\\Users\\patrick.carey\\Desktop\\RamRacing\\" + race + "_" + city + "_" + str(year) +".csv"
else:
    id_list = data[(data["Event"] == race) & (data["Year"] == year) & (data["event_location"]==city) & (data["Category"]==category)]["Registrant ID"]
    fileName = "C:\\Users\\patrick.carey\\Desktop\\RamRacing\\"+race + "_" + city + "_" + str(year) + "_" + category +".csv"

df = data[data["event_date"] < date[0]].copy()
df = df.drop("Count", 1)

g = df[["Registrant ID", 'Registration Category',"Category","Event","Kids"]].copy()
#dummy variable creation
#get dummies will remove original, so create a copy first
# g["Event_"] = g["Event"]
# g["Category_"] = g["Category"]
df["Gender"] = df["Registrant Gender"]
df = pd.get_dummies(df, columns=["Gender"],dummy_na=False, prefix="",prefix_sep="")
# g["Kids_"] = g["Kids"]
# data["Registration Event Name_"] = data["Registration Event Name"]
g = pd.get_dummies(g, columns=["Category","Event","Kids"],dummy_na=False, prefix="",prefix_sep="")
#add 5 mile and 5k to triple nickel
g['5 MILE'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5 MILE'])
g['5K'] = np.where(g['Registration Category'].str.contains("TRIPLE NICKEL"), 1,g['5K'])
g = g.drop('Registration Category',1)
g = g.groupby("Registrant ID", as_index=False).sum()
w = weather[(weather["event_location"] == city) & (weather["event_date"] == date[0])].copy()

df = df[df["Category"].isin(relevantList)]
df = df.sort_values(by="event_date").drop_duplicates(subset="Registrant ID", keep="last")
df = pd.merge(df, g, on="Registrant ID")
df = pd.merge(df, w, on="event_location")

df["last_event_days_back"] = (df["event_date_y"] - df["event_date_x"])/1
df["Dependent Variable"] = np.where(df["Registrant ID"].isin(id_list), 1,0)

df = df.drop(["User Email","User First Name","User Last Name","Registrant Email","Registrant First Name","Registrant Last Name","Registrant Gender","Registrant Birthdate"],1)
df = df.drop(["fees_first","fees_first_until","fees_second","fees_second_until","fees_third","fees_third_until","fees_fourth","fees_fourth_until"],1)
df = df.drop(["Age","Registrant Zipcode","distance_qty"],1)

df.to_csv(fileName, index=False)

In [ ]:
# df.loc["Chicago, IL"]["Mean TemperatureF"] = df.loc['Chicago, IL']["event_date"].map(ChicagoWeather["Mean TemperatureF"])

In [ ]:
# Training before 2015, testing = 2015

In [ ]:
df = data[(data["event_date"].isin([pd.Timestamp("2014-11-09"),pd.Timestamp("2015-11-08")])) & (data["Event"]=="HOT CHOCOLATE")&(data["Category"]=="15K")& ]

In [ ]:
expand_columns(data)
data.head(1)

In [ ]:
x = pd.to_datetime("2014-11-09") 
x.desribe()

In [ ]:
df2 = data.groupby('Registrant ID',as_index=False).sum()

In [ ]:
expand_columns(data)
data.head()

In [ ]:
# imports
import pandas as pd
import matplotlib.pyplot as plt

# this allows plots to appear directly in the notebook
%matplotlib inline

# visualize the relationship between the features and the response using scatterplots
fig, axs = plt.subplots(1, 3, sharey=True)
# data.plot(kind='scatter', x='Age', y='Count', ax=axs[0], figsize=(16, 8))
data.plot(kind='scatter',  x='Age', y='Category_15K', ax=axs[1])
# data.plot(kind='scatter', x='Newspaper', y='Sales', ax=axs[2])

In [ ]:
print(__doc__)


# Code source: Jaques Grobler
# License: BSD 3 clause


import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model

# Load the diabetes dataset
# diabetes = datasets.load_diabetes()


# Use only one feature
diabetes_X = diabetes.data[:, np.newaxis, 2]

# Split the data into training/testing sets
diabetes_X_train = diabetes_X[:-20]
diabetes_X_test = diabetes_X[-20:]

# Split the targets into training/testing sets
diabetes_y_train = diabetes.15K[:-20]
diabetes_y_test = diabetes.15K[-20:]

# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(diabetes_X_train, diabetes_y_train)

# The coefficients
print('Coefficients: \n', regr.coef_)
# The mean square error
print("Residual sum of squares: %.2f"
      % np.mean((regr.predict(diabetes_X_test) - diabetes_y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % regr.score(diabetes_X_test, diabetes_y_test))

# Plot outputs
plt.scatter(diabetes_X_test, diabetes_y_test,  color='black')
plt.plot(diabetes_X_test, regr.predict(diabetes_X_test), color='blue',
         linewidth=3)

plt.xticks(())
plt.yticks(())

plt.show()

In [ ]:
data["Week of increase fee"] = np.where((pd.to_datetime(data["Registration Date"]) - pd.to_datetime(data['fees_first_until'])) < np.timedelta64(7, 'D'),1,0)

In [ ]:
# # ID_Dict = data[["Registrant First Name","Registrant Last Name","Registrant Birthdate","Registrant ID"]]
# # ID_Dict = {tuple(x[:3]):x[3] for x in ID_Dict.values}

# data["TempID"] = zip(data[["Registrant First Name","Registrant Last Name","Registrant Birthdate"]])
# ID_Dict = data.set_index("TempID")["Registrant ID"].to_dict()
# data["TempID2"] = data["TempID"].map(ID_Dict)

In [ ]:
# data[data.duplicated(subset=["Registrant First Name","Registrant Last Name","Registrant Birthdate"], keep=False) ].sort_values(by=["Registrant First Name","Registrant Last Name","Registrant Birthdate"]).describe()

In [ ]:
# data[data.duplicated(subset=["Registrant First Name","Registrant Last Name","Registrant Birthdate","Registrant ID", "Registrant Gender"], keep=False)].sort_values(by=["Registrant First Name","Registrant Last Name","Registrant Birthdate"]).describe()

In [ ]:
expand_columns(data) #same registrant at events
data[data.duplicated(subset=["Registrant First Name","Registrant Last Name","Registrant Birthdate","Registration Event Name"], keep=False)].sort_values(by=["Registrant First Name","Registrant Last Name","Registrant Birthdate"]).to_csv("duplicated_event.csv")

In [ ]:
data[data["User ID"]=="524467a0-2728-44e6-aefb-5ba3c0a86524"].describe()

In [ ]:
pd.set_option('display.max_columns',len(data.columns))
# data[data['Registration Category']=="1.5 MILE FAMILY WALK"].head()
# data["results_net_pace_Minutes"].head()
data.head()


In [ ]:

#fill in blank registrant ID with User ID
#data[data["Registrant ID"].isnull() & (data["User First Name"] != data["Registrant First Name"]) & data["Registrant First Name"].notnull()].sort_values(by="User ID")
data["Registrant ID"] = data["Registrant ID"].fillna("FILL_NULL_w_USER_ID:"+data["User ID"])

#removed duplicated registrant event date rows, example is 2 identical entries but one does not have results_net_pace
    # also back in 2009 database was new and some errors in entries caused duplication, so the legend goes
data = data.sort_values(by="results_net_pace", na_position="last").drop_duplicates(subset=["Registrant ID",'event_date',"Registration Event Name","Registration Category"], keep="first").sort_values(by="Registrant ID")



In [ ]:
pd.set_option('display.max_columns',len(data.columns))
data[data["Event"] =="HOT CHOCOLATE"].head()

In [ ]:
# data = data.drop('Registrant Zipcode', 1) #drop zip due to SAS proc import error
data.to_csv(r"F:\SAS_VA\RAM_Transaction_2016_5_12.csv", index=False)

In [ ]:
pd.set_option('display.max_columns',len(data.columns))
data[data["fees_event"] < 0].head()
# data[data["fees_event"] < 0].unique()

In [ ]:
data[data["Registrant ID"]=="4d8aba6a-1f00-41f4-ae07-0c4f7f000001"]